In [1]:
def get_station_data(site_no="0702200"):
    # importing the requests library 
    import requests 
    import xml.etree.ElementTree as ET
    from zipfile import ZipFile
    import io
    from io import BytesIO
    # defining the api-endpoint  
    URL = "https://cida.usgs.gov/"
    API_ENDPOINT = "https://cida.usgs.gov/sediment/data"
    SITE_NO = "07022000"
    ogcfilter = ET.Element("ogc:Filter",attrib={"xmlns:ogc":"http://www.opengis.net/ogc"})
    property_is = ET.SubElement(ogcfilter, "ogc:PropertyIsEqualTo", attrib={"matchCase":"true"})
    ET.SubElement(property_is, "ogc:PropertyName").text = "SITE_NO"
    ET.SubElement(property_is, "ogc:Literal").text = SITE_NO
    ogcfilter_string = ET.tostring(ogcfilter).decode()  
    data = {'email' : 'user@usace.army.mil',
            'format': 'tsv',
            'directDownload':'true',
            'dataTypes': 'sites_data_daily_discrete_',
            'dailyFilter' : ogcfilter_string,
            'discreteFilter' : ogcfilter_string
           }

    # sending post request and saving response as response object 
    search = requests.post(url = API_ENDPOINT, data = data, allow_redirects=True)  
    download = requests.get(url=URL+search.text)
    zipfile = ZipFile(BytesIO(download._content))
    return zipfile
zipfile = get_station_data()

In [2]:
def get_comments(filename):
    ncomments=0
    comments=''
    with zipfile.open(filename) as f:
        commentline = f.readline().decode()
        while commentline:
            if commentline[0] == '#' or commentline[:2] == r'"#':
                ncomments +=1
                comments += commentline.strip(r'"').strip('#')
                commentline=f.readline().decode()
            else:
                commentline=None
    return (ncomments, comments)
(ncomments, comments) = get_comments('discrete_data.tsv')

In [3]:
import pandas
from beakerx import *
discrete_data = pandas.read_table(zipfile.open('discrete_data.tsv'),header=ncomments).dropna(axis=1,how='all')
discrete_data

,SITE_NO,STATION_NM,DATETIME,DCOMMENT,ICOMMENT,SSC,DAILYFLOW,INSTFLOW,GH,P2,...,TURB76,TURB63675,TURB63676,TEMPC,TEMPAIRC,SC,SCLAB,DSS,PH,PHLAB
0,7022000,"MISSISSIPPI RIVER AT THEBES, IL",1973-01-30 10:30:00,G,S,306,369000,NaN,NaN,NaN,...,NaN,NaN,NaN,1.5,NaN,435.0,NaN,NaN,7.8,NaN
1,7022000,"MISSISSIPPI RIVER AT THEBES, IL",1973-02-21 10:50:00,G,S,436,237000,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,430.0,NaN,NaN,7.7,NaN
2,7022000,"MISSISSIPPI RIVER AT THEBES, IL",1973-06-21 11:00:00,G,S,468,408000,NaN,NaN,NaN,...,NaN,NaN,NaN,25.5,NaN,435.0,NaN,310.0,7.8,NaN
3,7022000,"MISSISSIPPI RIVER AT THEBES, IL",1973-07-18 09:30:00,G,S,227,179000,NaN,NaN,NaN,...,NaN,NaN,NaN,28.0,NaN,640.0,NaN,370.0,7.8,NaN
4,7022000,"MISSISSIPPI RIVER AT THEBES, IL",1973-08-14 10:00:00,G,S,211,208000,NaN,NaN,NaN,...,NaN,NaN,NaN,26.0,NaN,545.0,NaN,330.0,8.1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,7022000,"MISSISSIPPI RIVER AT THEBES, IL",2019-04-29 16:20:00,NaN,NaN,166,564000,550000.0,36.90,NaN,...,NaN,NaN,45,14.6,19.5,443.0,482.0,NaN,8.0,8.3
733,7022000,"MISSISSIPPI RIVER AT THEBES, IL",2019-04-29 16:35:00,NaN,NaN,167,564000,550000.0,36.94,NaN,...,NaN,NaN,NaN,14.7,NaN,NaN,NaN,NaN,NaN,NaN
734,7022000,"MISSISSIPPI RIVER AT THEBES, IL",2019-05-08 12:50:00,NaN,NaN,324,786000,812000.0,41.78,NaN,...,NaN,NaN,162,15.5,20.9,329.0,377.0,NaN,7.5,8.2
735,7022000,"MISSISSIPPI RIVER AT THEBES, IL",2019-05-28 15:30:00,NaN,NaN,349,779000,782000.0,41.27,NaN,...,NaN,NaN,146,19.7,31.0,391.0,395.0,NaN,7.6,8.1


In [4]:
%matplotlib ipympl
import matplotlib
from matplotlib import pyplot as plt
plt.figure()
plt.scatter(discrete_data.DAILYFLOW,discrete_data.SSC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
(ncomments, comments) = get_comments('daily_data.tsv')
daily_data = pandas.read_table(zipfile.open('daily_data.tsv'),header=ncomments).dropna(axis=1,how='all')
daily_data

,agency_cd,site_no,datetime,DAILY_FLOW,DAILY_FLOW_QUAL,DAILY_SSC,DAILY_SSC_QUAL,DAILY_SSL,DAILY_SSL_QUAL
0,USGS,7022000,1933-04-01,191000,A,NaN,NaN,NaN,NaN
1,USGS,7022000,1933-04-02,224000,A,NaN,NaN,NaN,NaN
2,USGS,7022000,1933-04-03,257000,A,NaN,NaN,NaN,NaN
3,USGS,7022000,1933-04-04,279000,A,NaN,NaN,NaN,NaN
4,USGS,7022000,1933-04-05,298000,A,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
30964,USGS,7022000,2020-01-27,336000,P,NaN,NaN,NaN,NaN
30965,USGS,7022000,2020-01-28,331000,P,NaN,NaN,NaN,NaN
30966,USGS,7022000,2020-01-29,335000,P,NaN,NaN,NaN,NaN
30967,USGS,7022000,2020-01-30,338000,P,NaN,NaN,NaN,NaN


In [6]:
plt.figure()
plt.scatter(daily_data.DAILY_FLOW,daily_data.DAILY_SSC)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …